<a href="https://colab.research.google.com/github/JefNtungila/DS-Unit-4-Sprint-1-NLP/blob/master/LS_DS_415_Sprint_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
pip install squarify

In [2]:
pip install spacy

In [3]:
#RESTART RUNTIME

!python -m spacy download en_core_web_md 

     |████████████████████████████████| 95.4MB 94.5MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=bab9f755a98a03ba9ebf9d8b234e7c74e080d5302c2509a0e4be3a7dea739e8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-o3j0qqwa/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [27]:
pip install scattertext

     |████████████████████████████████| 6.7MB 2.7MB/s 


In [4]:
from google.colab import files
uploaded = files.upload()

Saving review_sample.json to review_sample.json


In [0]:
import pandas as pd

yelp = pd.read_json('review_sample.json', lines=True)

In [57]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [58]:
pip install regex

In [0]:
import regex as re


def tokenize(text):
    """Parses a string into a list of semantic units (words)

    Args:
        text (str): The string that the function will tokenize.

    Returns:
        list: tokens parsed out by the mechanics of your choice
    """
    
    tokens = re.sub(r'[^a-zA-Z ^0-9]', '', text)
    tokens = tokens.lower().split()
    
    return tokens

In [0]:
yelp['tokens'] = yelp['text'].apply(tokenize)

In [61]:
yelp[['tokens']].head()

,tokens
0,"[beware, fake, fake, fakewe, also, own, a, sma..."
1,"[came, here, for, lunch, togo, service, was, q..."
2,"[ive, been, to, vegas, dozens, of, times, and,..."
3,"[we, went, here, on, a, night, where, they, cl..."
4,"[35, to, 4, starsnot, bad, for, the, price, 12..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

###part 1

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(stop_words='english', min_df = 0.05, max_df= 0.90)

vect.fit(yelp['text'].tolist())

sparse_dtm = vect.transform(yelp['text'].tolist())

dtm = pd.DataFrame(sparse_dtm.todense(), columns=vect.get_feature_names())

In [63]:
dtm.head()

,10,amazing,area,asked,away,awesome,bad,bar,best,better,big,bit,came,check,cheese,chicken,clean,come,coming,customer,day,definitely,delicious,did,didn,different,dinner,don,drink,drinks,eat,excellent,experience,favorite,feel,food,fresh,friendly,friends,going,...,prices,quality,really,recommend,restaurant,right,room,said,salad,sauce,say,server,service,small,staff,stars,super,sure,table,taste,thing,things,think,time,times,told,took,tried,try,ve,vegas,wait,want,wanted,wasn,way,went,work,worth,years
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,2,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


###part 2

In [64]:
# Instantiate
from sklearn.neighbors import NearestNeighbors


nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [65]:
nn.kneighbors([dtm.iloc[0]])

(array([[0.        , 2.        , 2.        , 2.44948974, 2.44948974]]),
 array([[   0, 3428, 3133,  797, 3390]]))

In [66]:
yelp['text'].tolist()[50]

'One of the top restaurants in Montreal. We always make a point to eat here when we visit.  Go hungry because the food is decadent Quebecois fare. Martin Picard (the chef and owner) embraces gluttony, and so should you if you eat at Au Pied de Cochon.'

In [67]:
fake_review = [''''One of the worst restaurants in Montreal. The food is never properly cooked.''']

fake_review_right_length.append(fake_review)

NameError: ignored

In [0]:
# Query for Sim of Random doc to BBC
new = tfidf.transform(fake_review)

nn.kneighbors(new.todense())

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

# create the transformer
vect = CountVectorizer()

# tokenize and build vocab
vect.fit(yelp['text'])

# transform text
dtm = vect.transform(yelp['text'])


In [69]:
print(vect.get_feature_names())

['00', '000', '001695', '007', '00a', '00am', '00ish', '00pm', '01', '011802', '02', '02022015', '02pm', '03', '04', '05', '05pm', '06', '07', '08', '09', '0d', '0mph', '10', '100', '1000', '1001', '100k', '100lb', '100s', '100th', '100x', '101', '102', '1026', '1030am', '104', '105', '106', '106lb', '107', '108', '109', '1096', '1099', '10am', '10k', '10min', '10mins', '10news', '10p', '10pm', '10th', '10times', '10x', '10years', '10yrs', '11', '110', '110th', '111', '1115a', '1130', '115', '117', '119', '11a', '11am', '11ish', '11pm', '12', '120', '1200', '1221', '123', '123lbs', '1245p', '125', '1260', '129', '12am', '12k', '12months', '12oz', '12pk', '12pm', '12th', '12か所のコーナーのステアリングさばきは実にお見事でとてもいい体験になりました', '13', '130', '131', '135', '139', '13th', '14', '140', '1400', '14000', '141', '144', '1440', '145', '149', '14oz', '14th', '15', '150', '1500', '150k', '155', '15am', '15ish', '15k', '15min', '15mins', '15pm', '15th', '15をくぐる形', '15を南へ15', '16', '160', '1611', '165', '167', '1

###RandomForest

In [0]:
X = dtm
y = yelp['stars']

In [71]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(8000, 27588) (8000,) (2000, 27588) (2000,)


In [0]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state = 42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# from sklearn.metrics import accuracy_score

# print(accuracy_score(y_test, y_pred)



In [73]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))


0.4345


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [0]:

import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_md")

tokenizer = Tokenizer(nlp.vocab)
tokens = []

""" Update those tokens w/o stopwords"""
for doc in tokenizer.pipe(yelp['text'], batch_size=500):
    doc_tokens = []
    
    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.text.lower())
    tokens.append(doc_tokens)
            
yelp['tokens2'] = tokens

In [0]:


id2word = corpora.Dictionary(yelp['tokens2'])
corpus = [id2word.doc2bow(text) for text in yelp['tokens2']]

In [105]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   passes=10,
                   workers=4

lda.print_topics()

[(0,
  '0.019*"\n\n" + 0.016*" " + 0.006*"good" + 0.006*"like" + 0.004*"place" + 0.004*"got" + 0.003*"time" + 0.003*"don\'t" + 0.003*"didn\'t" + 0.002*"told"'),
 (1,
  '0.023*"\n\n" + 0.008*"food" + 0.007*"good" + 0.007*" " + 0.007*"chicken" + 0.007*"like" + 0.006*"great" + 0.005*"place" + 0.005*"little" + 0.004*"\n"'),
 (2,
  '0.021*" " + 0.018*"\n\n" + 0.007*"place" + 0.006*"food" + 0.006*"\n" + 0.005*"it\'s" + 0.004*"good" + 0.003*"told" + 0.003*"service" + 0.003*"didn\'t"'),
 (3,
  '0.028*"\n\n" + 0.006*"it\'s" + 0.005*"like" + 0.005*"place" + 0.004*"good" + 0.004*" " + 0.004*"people" + 0.004*"got" + 0.004*"food" + 0.003*"went"'),
 (4,
  '0.011*"\n" + 0.007*"time" + 0.007*"\n\n" + 0.006*"service" + 0.006*"place" + 0.005*"food" + 0.005*"good" + 0.005*"best" + 0.004*"like" + 0.004*"i\'ve"'),
 (5,
  '0.068*" " + 0.009*"\n\n" + 0.008*"\n" + 0.006*"good" + 0.005*"food" + 0.005*"like" + 0.005*"time" + 0.005*"great" + 0.004*"place" + 0.004*"service"'),
 (6,
  '0.026*"\n" + 0.007*"\n\n" + 

In [106]:

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]#cleaning
topics = [' '.join(t[0:5]) for t in words]
for t in topics:
    print(t)
    print('\n')



   good like place




 food good   chicken


  

 place food 





 it's like place good



 time 

 service place


  

 
 good food



 

 place got it's




   de good la




 great place food like


  

 
 good service


  

 food place service




 like don't i've time


food best service place love


  

 great place like




 great place food best




### Key terms found that are used with high star reviews and others used with lower star reviews

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)